In [1]:
import torch
import pyro
import pyro.distributions as dist
from pyro.infer import SVI, Trace_ELBO
from pyro.optim import Adam

In [2]:
print(torch.__version__)
print(pyro.__version__)

1.3.0.post2
0.5.1


In [3]:
pyro.enable_validation(True)
pyro.clear_param_store()

In [4]:
data = torch.cat((torch.zeros(9), torch.ones(7), torch.empty(4).fill_(2.)))

In [5]:
data

tensor([0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 1., 1., 1., 1., 1., 1., 2., 2.,
        2., 2.])

In [6]:
def model(data):
    alpha = torch.tensor(6.0)
    beta = torch.tensor(10.0)
    pay_probs = pyro.sample('pay_probs', dist.Beta(alpha, beta).expand([3]).independent(1))
    normalized_pay_probs = pay_probs / torch.sum(pay_probs)
    
    for i in range(len(data)):
        pyro.sample('obs_{%d}' % (i), dist.Categorical(probs=normalized_pay_probs), obs=data[i])

In [7]:
def guide(data):
    alphas = pyro.param('alphas', torch.tensor(6.).expand(3), constraint=constraints.positive)
    betas = pyro.param('betas', torch.tensor(10.).expand(3), constraint=constraints.positive) 

    pyro.sample('pay_probs', dist.Beta(alphas, betas).independent(1))

In [8]:
def print_progress():
    alphas = pyro.param("alphas")
    betas = pyro.param("betas")

    if torch.cuda.is_available():
        alphas.cuda()
        betas.cuda()

    means = alphas / (alphas + betas)
    normalized_means = means / torch.sum(means)
    factors = betas / (alphas * (1.0 + alphas + betas))
    stdevs = normalized_means * torch.sqrt(factors)

    tiger_pays_string = "probability Tiger pays: {0:.3f} +/- {1:.2f}".format(normalized_means[0], stdevs[0])
    jason_pays_string = "probability Jason pays: {0:.3f} +/- {1:.2f}".format(normalized_means[1], stdevs[1])
    james_pays_string = "probability James pays: {0:.3f} +/- {1:.2f}".format(normalized_means[2], stdevs[2])
    print("[", step, "|", tiger_pays_string, "|", jason_pays_string, "|", james_pays_string, "]")

In [9]:
from torch.distributions import constraints

In [10]:
pyro.clear_param_store()
adam_params = {"lr": 0.0005}
optimizer = Adam(adam_params)
svi = SVI(model, guide, optimizer, loss=Trace_ELBO())

n_steps = 2501
for step in range(n_steps):
    svi.step(data)
    if step % 100 == 0:
        print_progress()

[ 0 | probability Tiger pays: 0.333 +/- 0.10 | probability Jason pays: 0.333 +/- 0.10 | probability James pays: 0.333 +/- 0.10 ]
[ 100 | probability Tiger pays: 0.341 +/- 0.10 | probability Jason pays: 0.335 +/- 0.10 | probability James pays: 0.325 +/- 0.10 ]
[ 200 | probability Tiger pays: 0.348 +/- 0.11 | probability Jason pays: 0.335 +/- 0.10 | probability James pays: 0.316 +/- 0.10 ]
[ 300 | probability Tiger pays: 0.353 +/- 0.11 | probability Jason pays: 0.336 +/- 0.10 | probability James pays: 0.311 +/- 0.10 ]
[ 400 | probability Tiger pays: 0.361 +/- 0.11 | probability Jason pays: 0.337 +/- 0.10 | probability James pays: 0.302 +/- 0.10 ]
[ 500 | probability Tiger pays: 0.365 +/- 0.11 | probability Jason pays: 0.338 +/- 0.10 | probability James pays: 0.297 +/- 0.10 ]
[ 600 | probability Tiger pays: 0.367 +/- 0.11 | probability Jason pays: 0.340 +/- 0.10 | probability James pays: 0.293 +/- 0.10 ]
[ 700 | probability Tiger pays: 0.370 +/- 0.11 | probability Jason pays: 0.340 +/- 0.